# Model Experimenting
This notebook will work as an experiment on how well different ML models do on historical data for different stocks.

## Importing

In [ ]:
from typing import Union
import numpy as np
import sys

from pathlib import Path
sys.path.append(str(Path("..").resolve()))

from live_trader.ml_model import ML_Pipeline, brier, AI_strategy, attention_bilstm_strategy

/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# Tensorflow
import tensorflow as tf

from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Bidirectional,
    Attention, LayerNormalization, Add, GlobalAveragePooling1D, 
    Conv1D, MultiHeadAttention, Reshape, Lambda, GRU
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

## Testing our models that are already made

### Basic LSTM

In [3]:
side, _ = await AI_strategy("GOOG")
print(f"GOOG: {side}")

Epoch 1/20
52/52 - 2s - 41ms/step - brier: 0.2532 - loss: 0.6995 - roc_auc: 0.5040 - val_brier: 0.2786 - val_loss: 0.7276 - val_roc_auc: 0.5209
Epoch 2/20
52/52 - 1s - 14ms/step - brier: 0.2510 - loss: 0.6944 - roc_auc: 0.5083 - val_brier: 0.2965 - val_loss: 0.7567 - val_roc_auc: 0.5390
Epoch 3/20
52/52 - 1s - 13ms/step - brier: 0.2499 - loss: 0.6914 - roc_auc: 0.5189 - val_brier: 0.2700 - val_loss: 0.7104 - val_roc_auc: 0.5446
Epoch 4/20
52/52 - 1s - 14ms/step - brier: 0.2498 - loss: 0.6894 - roc_auc: 0.5230 - val_brier: 0.3011 - val_loss: 0.7641 - val_roc_auc: 0.5403
Epoch 5/20
52/52 - 1s - 13ms/step - brier: 0.2492 - loss: 0.6915 - roc_auc: 0.5007 - val_brier: 0.2960 - val_loss: 0.7516 - val_roc_auc: 0.5439
Epoch 6/20
52/52 - 1s - 13ms/step - brier: 0.2488 - loss: 0.6870 - roc_auc: 0.5235 - val_brier: 0.3068 - val_loss: 0.7734 - val_roc_auc: 0.5446
Epoch 7/20
52/52 - 1s - 14ms/step - brier: 0.2498 - loss: 0.6879 - roc_auc: 0.5294 - val_brier: 0.3024 - val_loss: 0.7637 - val_roc_auc:

In [4]:
side, _ = await AI_strategy("AAPL")
print(f"AAPL: {side}")

Epoch 1/20
52/52 - 2s - 40ms/step - brier: 0.2536 - loss: 0.7034 - roc_auc: 0.4993 - val_brier: 0.2499 - val_loss: 0.6960 - val_roc_auc: 0.4942
Epoch 2/20
52/52 - 1s - 13ms/step - brier: 0.2517 - loss: 0.6932 - roc_auc: 0.5355 - val_brier: 0.2487 - val_loss: 0.6918 - val_roc_auc: 0.5256
Epoch 3/20
52/52 - 1s - 13ms/step - brier: 0.2508 - loss: 0.6920 - roc_auc: 0.5279 - val_brier: 0.2482 - val_loss: 0.6918 - val_roc_auc: 0.5166
Epoch 4/20
52/52 - 1s - 14ms/step - brier: 0.2512 - loss: 0.6957 - roc_auc: 0.5290 - val_brier: 0.2494 - val_loss: 0.6918 - val_roc_auc: 0.5353
Epoch 5/20
52/52 - 1s - 13ms/step - brier: 0.2513 - loss: 0.6900 - roc_auc: 0.5444 - val_brier: 0.2490 - val_loss: 0.6945 - val_roc_auc: 0.5023
Epoch 6/20
52/52 - 1s - 13ms/step - brier: 0.2534 - loss: 0.6957 - roc_auc: 0.5436 - val_brier: 0.2501 - val_loss: 0.6913 - val_roc_auc: 0.5432
Epoch 7/20
52/52 - 1s - 14ms/step - brier: 0.2526 - loss: 0.6914 - roc_auc: 0.5474 - val_brier: 0.2485 - val_loss: 0.6935 - val_roc_auc:

In [5]:
side, _ = await AI_strategy("MCFT")
print(f"MCFT: {side}")

Epoch 1/20
52/52 - 2s - 44ms/step - brier: 0.2546 - loss: 0.6985 - roc_auc: 0.5213 - val_brier: 0.2462 - val_loss: 0.6991 - val_roc_auc: 0.4764
Epoch 2/20
52/52 - 1s - 14ms/step - brier: 0.2517 - loss: 0.6925 - roc_auc: 0.5346 - val_brier: 0.2502 - val_loss: 0.7022 - val_roc_auc: 0.4793
Epoch 3/20
52/52 - 1s - 13ms/step - brier: 0.2513 - loss: 0.6898 - roc_auc: 0.5443 - val_brier: 0.2523 - val_loss: 0.7050 - val_roc_auc: 0.4804
Epoch 4/20
52/52 - 1s - 14ms/step - brier: 0.2506 - loss: 0.6880 - roc_auc: 0.5572 - val_brier: 0.2559 - val_loss: 0.7075 - val_roc_auc: 0.4772
Epoch 5/20
52/52 - 1s - 13ms/step - brier: 0.2499 - loss: 0.6854 - roc_auc: 0.5616 - val_brier: 0.2575 - val_loss: 0.7147 - val_roc_auc: 0.4749
Epoch 6/20
52/52 - 1s - 13ms/step - brier: 0.2517 - loss: 0.6859 - roc_auc: 0.5679 - val_brier: 0.2543 - val_loss: 0.7068 - val_roc_auc: 0.4820
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
MCFT: SideSignal.HOLD


### attention bilstm

In [6]:
side, _ = await attention_bilstm_strategy("GOOG")
print(f"GOOG: {side}")

Epoch 1/20
52/52 - 4s - 81ms/step - brier: 0.2664 - loss: 0.7234 - roc_auc: 0.5257 - val_brier: 0.2952 - val_loss: 0.7658 - val_roc_auc: 0.5064
Epoch 2/20
52/52 - 2s - 32ms/step - brier: 0.2600 - loss: 0.7275 - roc_auc: 0.4911 - val_brier: 0.5455 - val_loss: 1.8710 - val_roc_auc: 0.4888
Epoch 3/20
52/52 - 3s - 67ms/step - brier: 0.2664 - loss: 0.7281 - roc_auc: 0.5027 - val_brier: 0.3216 - val_loss: 0.9846 - val_roc_auc: 0.4884
Epoch 4/20
52/52 - 2s - 32ms/step - brier: 0.2572 - loss: 0.7347 - roc_auc: 0.5174 - val_brier: 0.5756 - val_loss: 2.7356 - val_roc_auc: 0.5000
Epoch 5/20
52/52 - 2s - 33ms/step - brier: 0.2508 - loss: 0.6958 - roc_auc: 0.5136 - val_brier: 0.3392 - val_loss: 0.8511 - val_roc_auc: 0.5011
Epoch 6/20
52/52 - 2s - 34ms/step - brier: 0.2484 - loss: 0.6920 - roc_auc: 0.5058 - val_brier: 0.3534 - val_loss: 0.8830 - val_roc_auc: 0.4898
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
GOOG: SideSignal.HOLD


In [7]:
side, _ = await attention_bilstm_strategy("AAPL")
print(f"AAPL: {side}")

Epoch 1/20
52/52 - 5s - 88ms/step - brier: 0.2745 - loss: 0.7551 - roc_auc: 0.5100 - val_brier: 0.2726 - val_loss: 0.7259 - val_roc_auc: 0.4831
Epoch 2/20
52/52 - 2s - 33ms/step - brier: 0.2754 - loss: 0.7495 - roc_auc: 0.5036 - val_brier: 0.2702 - val_loss: 0.7187 - val_roc_auc: 0.5080
Epoch 3/20
52/52 - 2s - 32ms/step - brier: 0.2585 - loss: 0.7100 - roc_auc: 0.5061 - val_brier: 0.2451 - val_loss: 0.7109 - val_roc_auc: 0.5400
Epoch 4/20
52/52 - 2s - 32ms/step - brier: 0.2566 - loss: 0.7047 - roc_auc: 0.5118 - val_brier: 0.2477 - val_loss: 0.7032 - val_roc_auc: 0.5324
Epoch 5/20
52/52 - 2s - 32ms/step - brier: 0.2539 - loss: 0.6979 - roc_auc: 0.5200 - val_brier: 0.2456 - val_loss: 0.6974 - val_roc_auc: 0.5357
Epoch 6/20
52/52 - 2s - 32ms/step - brier: 0.2511 - loss: 0.6945 - roc_auc: 0.5215 - val_brier: 0.2499 - val_loss: 0.6947 - val_roc_auc: 0.5451
Epoch 7/20
52/52 - 2s - 32ms/step - brier: 0.2493 - loss: 0.6893 - roc_auc: 0.5251 - val_brier: 0.2446 - val_loss: 0.6909 - val_roc_auc:

In [8]:
side, _ = await attention_bilstm_strategy("MCFT")
print(f"MCFT: {side}")

Epoch 1/20
52/52 - 4s - 81ms/step - brier: 0.2839 - loss: 0.7952 - roc_auc: 0.5068 - val_brier: 0.2797 - val_loss: 0.7248 - val_roc_auc: 0.5370
Epoch 2/20
52/52 - 2s - 33ms/step - brier: 0.2772 - loss: 0.7515 - roc_auc: 0.5048 - val_brier: 0.2721 - val_loss: 0.7278 - val_roc_auc: 0.5338
Epoch 3/20
52/52 - 2s - 32ms/step - brier: 0.2669 - loss: 0.7313 - roc_auc: 0.5140 - val_brier: 0.2710 - val_loss: 0.7116 - val_roc_auc: 0.5217
Epoch 4/20
52/52 - 2s - 32ms/step - brier: 0.2569 - loss: 0.7101 - roc_auc: 0.5020 - val_brier: 0.2617 - val_loss: 0.7037 - val_roc_auc: 0.4815
Epoch 5/20
52/52 - 2s - 32ms/step - brier: 0.2507 - loss: 0.6918 - roc_auc: 0.5378 - val_brier: 0.2575 - val_loss: 0.6977 - val_roc_auc: 0.5157
Epoch 6/20
52/52 - 2s - 32ms/step - brier: 0.2508 - loss: 0.6926 - roc_auc: 0.5341 - val_brier: 0.2623 - val_loss: 0.7001 - val_roc_auc: 0.5080
Epoch 7/20
52/52 - 2s - 32ms/step - brier: 0.2501 - loss: 0.6900 - roc_auc: 0.5345 - val_brier: 0.2662 - val_loss: 0.7141 - val_roc_auc:

Both attention_bilstm and basic_lstm are not good models. Therefore, we will try out other models as well.

## Modelling

### Temporal Convolutional Network (TCN-lite)

In [9]:
def build_tcn_lite(X_train_seq: Union[np.ndarray, list]) -> Model:
    """
    Builds a lightweight Temporal Convolutional Network (TCN-style)
    for noisy financial time series classification.

    Designed to be robust to non-stationarity and overfitting.

    Args:
        X_train_seq (array-like):
            Training sequences of shape (n_samples, time_steps, n_features)

    Returns:
        Compiled Keras Model
    """
    n_features = X_train_seq.shape[2]

    inputs = Input(shape=(None, n_features))

    x = Conv1D(
        filters=32,
        kernel_size=3,
        padding="causal",
        activation="relu"
    )(inputs)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)

    x = Conv1D(
        filters=16,
        kernel_size=3,
        padding="causal",
        activation="relu"
    )(x)
    x = LayerNormalization()(x)

    x = GlobalAveragePooling1D()(x)

    x = Dense(16, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs, name="tcn_lite")

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[
            AUC(name="auc"),
            brier
        ]
    )

    return model

### PatchTST

In [10]:
def build_patchtst_lite(
    X_train_seq: Union[np.ndarray, list],
    patch_len: int = 16,
    d_model: int = 64,
    num_heads: int = 4,
    ff_dim: int = 128,
    dropout: float = 0.3
) -> Model:
    """
    Builds a lightweight PatchTST-style Transformer model for
    noisy financial time series classification.

    The model splits the time dimension into patches, embeds them,
    and applies a Transformer encoder for temporal modeling.

    Designed for robustness to non-stationarity and overfitting.

    Args:
        X_train_seq (array-like):
            Training sequences of shape (n_samples, time_steps, n_features)

        patch_len (int):
            Length of each temporal patch

        d_model (int):
            Transformer embedding dimension

        num_heads (int):
            Number of attention heads

        ff_dim (int):
            Feed-forward network size inside Transformer

        dropout (float):
            Dropout rate

    Returns:
        Compiled Keras Model
    """

    n_features = X_train_seq.shape[2]

    inputs = Input(shape=(None, n_features))

    # ---- Patch embedding ----
    # Split time dimension into non-overlapping patches
    def patchify(x):
        batch_size = tf.shape(x)[0]
        time_steps = tf.shape(x)[1]

        # Ensure at least one patch
        pad_len = tf.maximum(0, patch_len - time_steps)
        x = tf.pad(x, [[0, 0], [0, pad_len], [0, 0]])

        # Recompute after padding
        time_steps = tf.shape(x)[1]
        n_patches = time_steps // patch_len

        x = x[:, :n_patches * patch_len, :]
        x = tf.reshape(x, (batch_size, n_patches, patch_len * n_features))
        return x

    x = tf.keras.layers.Lambda(patchify, name="patchify")(inputs)

    x = Dense(d_model, activation="linear")(x)
    x = LayerNormalization()(x)

    # ---- Transformer Encoder Block ----
    attn_out = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=d_model // num_heads,
        dropout=dropout
    )(x, x)

    x = LayerNormalization()(x + attn_out)

    ff_out = Dense(ff_dim, activation="relu")(x)
    ff_out = Dropout(dropout)(ff_out)
    ff_out = Dense(d_model)(ff_out)

    x = LayerNormalization()(x + ff_out)

    # ---- Pooling & Head ----
    x = GlobalAveragePooling1D()(x)

    x = Dense(32, activation="relu")(x)
    x = Dropout(dropout)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs, name="patchtst_lite")

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[
            AUC(name="auc"),
            brier
        ]
    )

    return model



### GNN (Graph-NN)

In [11]:
class GraphMessagePassing(tf.keras.layers.Layer):
    """
    Simple graph message-passing layer with learned adjacency.
    """

    def __init__(self, hidden_dim: int, dropout: float = 0.0, **kwargs):
        super().__init__(**kwargs)
        self.hidden_dim = hidden_dim
        self.dropout = dropout

    def build(self, input_shape):
        # input_shape: (B, N_nodes, D)
        n_nodes = input_shape[1]

        self.adjacency = Dense(
            n_nodes,
            activation="tanh",
            name="learned_adjacency"
        )
        self.node_update = Dense(self.hidden_dim, activation="relu")
        self.norm = LayerNormalization()
        self.drop = Dropout(self.dropout)

        super().build(input_shape)

    def call(self, x):
        # x: (B, N, D)
        A = self.adjacency(x)          # (B, N, N)
        messages = tf.matmul(A, x)    # (B, N, D)
        x = self.node_update(messages)
        x = self.norm(x)
        return self.drop(x)


In [12]:
def build_gnn_lite(
    X_train_seq: Union[np.ndarray, list],
    hidden_dim: int = 32,
    gnn_layers: int = 2,
    dropout: float = 0.3
) -> Model:
    """
    Builds a lightweight Graph Neural Network (GNN-style) model
    for noisy financial time series classification.

    Nodes represent features (indicators).
    Edges are learned implicitly via feature interactions.

    Designed for robustness to:
    - Non-stationarity
    - Variable-length sequences
    - Small batch sizes

    Args:
        X_train_seq (array-like):
            Training sequences of shape (n_samples, time_steps, n_features)

        hidden_dim (int):
            Node embedding dimension

        gnn_layers (int):
            Number of graph message-passing layers

        dropout (float):
            Dropout rate

    Returns:
        Compiled Keras Model
    """

    n_features = X_train_seq.shape[2]

    inputs = Input(shape=(None, n_features))

    # --------------------------------------------------
    # Temporal aggregation
    # --------------------------------------------------
    # (B, T, F) → (B, F)
    x = GlobalAveragePooling1D(name="temporal_pool")(inputs)

    # Treat features as nodes
    # (B, F) → (B, F, 1)
    x = Lambda(lambda t: tf.expand_dims(t, axis=-1))(x)

    # --------------------------------------------------
    # GNN layers
    # --------------------------------------------------
    for i in range(gnn_layers):
        x = GraphMessagePassing(
            hidden_dim=hidden_dim,
            dropout=dropout,
            name=f"gnn_layer_{i}"
        )(x)

    # --------------------------------------------------
    # Graph pooling
    # --------------------------------------------------
    x = Lambda(lambda t: tf.reduce_mean(t, axis=1))(x)

    # --------------------------------------------------
    # Head
    # --------------------------------------------------
    x = Dense(32, activation="relu")(x)
    x = Dropout(dropout)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs, name="gnn_lite")

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[AUC(name="auc"), brier]
    )

    return model


### Neural Anomaly Detection

In [13]:
import keras
from keras import ops


class AutoencoderClassifierLite(keras.Model):
    """
    Autoencoder + Classifier with internal reconstruction loss.

    Keras 3–safe implementation using subclassed Model.
    """

    def __init__(
        self,
        n_features: int,
        latent_dim: int = 16,
        hidden_dim: int = 64,
        dropout: float = 0.3,
        recon_weight: float = 0.3,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.recon_weight = recon_weight

        # -------- Pooling --------
        self.pool = GlobalAveragePooling1D()

        # -------- Encoder --------
        self.enc_dense = Dense(hidden_dim, activation="relu")
        self.enc_norm = LayerNormalization()
        self.enc_drop = Dropout(dropout)
        self.latent = Dense(latent_dim, activation="linear")

        # -------- Decoder --------
        self.dec_dense = Dense(hidden_dim, activation="relu")
        self.dec_drop = Dropout(dropout)
        self.reconstruction = Dense(n_features, activation="linear")

        # -------- Classifier --------
        self.cls_dense = Dense(32, activation="relu")
        self.cls_drop = Dropout(dropout)
        self.output_head = Dense(1, activation="sigmoid")

    def call(self, inputs, training=False):
        # -------------------------
        # Pool input
        # -------------------------
        pooled = self.pool(inputs)

        # -------------------------
        # Encode
        # -------------------------
        x = self.enc_dense(pooled)
        x = self.enc_norm(x)
        x = self.enc_drop(x, training=training)

        latent = self.latent(x)

        # -------------------------
        # Decode (reconstruction)
        # -------------------------
        d = self.dec_dense(latent)
        d = self.dec_drop(d, training=training)
        recon = self.reconstruction(d)

        # -------------------------
        # Reconstruction loss
        # -------------------------
        diff = pooled - recon
        recon_loss = ops.mean(ops.square(diff))
        self.add_loss(self.recon_weight * recon_loss)

        # -------------------------
        # Classification
        # -------------------------
        c = self.cls_dense(latent)
        c = self.cls_drop(c, training=training)
        return self.output_head(c)


def build_autoencoder_classifier_lite(
    X_train_seq: Union[np.ndarray, list],
    latent_dim: int = 16,
    hidden_dim: int = 64,
    dropout: float = 0.3,
    recon_weight: float = 0.3
) -> keras.Model:
    """
    Builds an Autoencoder + Classifier model for
    neural anomaly detection in time series.

    Fully compatible with Keras 3 and existing pipelines.
    """

    n_features = X_train_seq.shape[2]

    model = AutoencoderClassifierLite(
        n_features=n_features,
        latent_dim=latent_dim,
        hidden_dim=hidden_dim,
        dropout=dropout,
        recon_weight=recon_weight,
        name="autoencoder_classifier_lite"
    )

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[AUC(name="auc"), brier]
    )

    return model


### CNN-GRU

In [14]:
def build_cnn_gru_lite(X_train_seq: Union[np.ndarray, list]) -> Model:
    """
    Builds a lightweight CNN-GRU model for
    noisy financial time series classification.

    Combines shallow temporal convolutions for
    local pattern extraction with a compact GRU
    layer for sequence modeling.

    Designed to be robust to non-stationarity
    and overfitting.

    Args:
        X_train_seq (array-like):
            Training sequences of shape
            (n_samples, time_steps, n_features)

    Returns:
        Compiled Keras Model
    """
    n_features = X_train_seq.shape[2]

    inputs = Input(shape=(None, n_features))

    # ---- CNN block ----
    x = Conv1D(filters=32, kernel_size=3, padding="same", activation="relu")(inputs)
    x = LayerNormalization()(x)
    x = Dropout(0.3)(x)

    x = Conv1D(
        filters=16,
        kernel_size=3,
        padding="same",
        activation="relu"
    )(x)
    x = LayerNormalization()(x)

    # ---- GRU block ----
    x = GRU(
        units=32,
        dropout=0.3
    )(x)

    # ---- Head ----
    x = Dense(16, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs, name="cnn_gru_lite")

    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss="binary_crossentropy",
        metrics=[
            AUC(name="auc"),
            brier
        ]
    )

    return model


## Training / Testing Models

### TCN-lite

In [15]:
symbol = "GOOG"
side, _ = await ML_Pipeline(build_tcn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 38ms/step - auc: 0.5274 - brier: 0.2533 - loss: 0.6947 - val_auc: 0.4940 - val_brier: 0.2598 - val_loss: 0.7019
Epoch 2/20
52/52 - 0s - 7ms/step - auc: 0.5016 - brier: 0.2503 - loss: 0.6950 - val_auc: 0.5016 - val_brier: 0.3027 - val_loss: 0.7765
Epoch 3/20
52/52 - 0s - 6ms/step - auc: 0.4970 - brier: 0.2491 - loss: 0.6916 - val_auc: 0.5012 - val_brier: 0.2703 - val_loss: 0.7186
Epoch 4/20
52/52 - 0s - 6ms/step - auc: 0.5009 - brier: 0.2481 - loss: 0.6923 - val_auc: 0.5209 - val_brier: 0.3552 - val_loss: 0.8912
Epoch 5/20
52/52 - 0s - 6ms/step - auc: 0.5464 - brier: 0.2476 - loss: 0.6867 - val_auc: 0.5196 - val_brier: 0.3182 - val_loss: 0.8076
Epoch 6/20
52/52 - 0s - 6ms/step - auc: 0.5035 - brier: 0.2489 - loss: 0.6923 - val_auc: 0.5179 - val_brier: 0.3016 - val_loss: 0.7751
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
GOOG: SideSignal.BUY


In [16]:
symbol = "AAPL"
side, _ = await ML_Pipeline(build_tcn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 46ms/step - auc: 0.4725 - brier: 0.2567 - loss: 0.7122 - val_auc: 0.4340 - val_brier: 0.2499 - val_loss: 0.6990
Epoch 2/20
52/52 - 0s - 6ms/step - auc: 0.4805 - brier: 0.2514 - loss: 0.6993 - val_auc: 0.4980 - val_brier: 0.2482 - val_loss: 0.6931
Epoch 3/20
52/52 - 0s - 6ms/step - auc: 0.5233 - brier: 0.2501 - loss: 0.6924 - val_auc: 0.5312 - val_brier: 0.2449 - val_loss: 0.6927
Epoch 4/20
52/52 - 0s - 6ms/step - auc: 0.5147 - brier: 0.2501 - loss: 0.6940 - val_auc: 0.4933 - val_brier: 0.2470 - val_loss: 0.6953
Epoch 5/20
52/52 - 0s - 6ms/step - auc: 0.5226 - brier: 0.2495 - loss: 0.6913 - val_auc: 0.5108 - val_brier: 0.2457 - val_loss: 0.6942
Epoch 6/20
52/52 - 0s - 6ms/step - auc: 0.5244 - brier: 0.2494 - loss: 0.6898 - val_auc: 0.5095 - val_brier: 0.2467 - val_loss: 0.6954
Epoch 7/20
52/52 - 0s - 6ms/step - auc: 0.5186 - brier: 0.2499 - loss: 0.6919 - val_auc: 0.5098 - val_brier: 0.2454 - val_loss: 0.6938
Epoch 8/20
52/52 - 0s - 6ms/step - auc: 0.5218 - brier

In [17]:
symbol = "MCFT"
side, _ = await ML_Pipeline(build_tcn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 38ms/step - auc: 0.5007 - brier: 0.2587 - loss: 0.7126 - val_auc: 0.5263 - val_brier: 0.2531 - val_loss: 0.6958
Epoch 2/20
52/52 - 0s - 7ms/step - auc: 0.4961 - brier: 0.2559 - loss: 0.7052 - val_auc: 0.4953 - val_brier: 0.2500 - val_loss: 0.6949
Epoch 3/20
52/52 - 0s - 6ms/step - auc: 0.5374 - brier: 0.2510 - loss: 0.6919 - val_auc: 0.4954 - val_brier: 0.2529 - val_loss: 0.6973
Epoch 4/20
52/52 - 0s - 6ms/step - auc: 0.5089 - brier: 0.2515 - loss: 0.6960 - val_auc: 0.5329 - val_brier: 0.2532 - val_loss: 0.6967
Epoch 5/20
52/52 - 0s - 6ms/step - auc: 0.5347 - brier: 0.2505 - loss: 0.6923 - val_auc: 0.5221 - val_brier: 0.2531 - val_loss: 0.6981
Epoch 6/20
52/52 - 0s - 6ms/step - auc: 0.5244 - brier: 0.2501 - loss: 0.6924 - val_auc: 0.4843 - val_brier: 0.2533 - val_loss: 0.7019
Epoch 7/20
52/52 - 0s - 6ms/step - auc: 0.5346 - brier: 0.2501 - loss: 0.6915 - val_auc: 0.4982 - val_brier: 0.2522 - val_loss: 0.6976
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━

### PathTST-lite

In [18]:
symbol = "GOOG"
side, _ = await ML_Pipeline(build_patchtst_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 3s - 65ms/step - auc: 0.4878 - brier: 0.2626 - loss: 0.7249 - val_auc: 0.4583 - val_brier: 0.3651 - val_loss: 0.9141
Epoch 2/20
52/52 - 1s - 10ms/step - auc: 0.5199 - brier: 0.2564 - loss: 0.7256 - val_auc: 0.5013 - val_brier: 0.5055 - val_loss: 1.4838
Epoch 3/20
52/52 - 0s - 8ms/step - auc: 0.5019 - brier: 0.2555 - loss: 0.6995 - val_auc: 0.4859 - val_brier: 0.2521 - val_loss: 0.7269
Epoch 4/20
52/52 - 0s - 8ms/step - auc: 0.5148 - brier: 0.2505 - loss: 0.6922 - val_auc: 0.4743 - val_brier: 0.2501 - val_loss: 0.6925
Epoch 5/20
52/52 - 0s - 8ms/step - auc: 0.5333 - brier: 0.2492 - loss: 0.6899 - val_auc: 0.4855 - val_brier: 0.2935 - val_loss: 0.7607
Epoch 6/20
52/52 - 0s - 8ms/step - auc: 0.5331 - brier: 0.2499 - loss: 0.6884 - val_auc: 0.5052 - val_brier: 0.2679 - val_loss: 0.7161
Epoch 7/20
52/52 - 1s - 12ms/step - auc: 0.5332 - brier: 0.2503 - loss: 0.6918 - val_auc: 0.4994 - val_brier: 0.2808 - val_loss: 0.7382
Epoch 8/20
52/52 - 0s - 8ms/step - auc: 0.5078 - bri

In [19]:
symbol = "AAPL"
side, _ = await ML_Pipeline(build_patchtst_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 4s - 74ms/step - auc: 0.4912 - brier: 0.2739 - loss: 0.7544 - val_auc: 0.5132 - val_brier: 0.2443 - val_loss: 0.6998
Epoch 2/20
52/52 - 1s - 10ms/step - auc: 0.5107 - brier: 0.2535 - loss: 0.7008 - val_auc: 0.4875 - val_brier: 0.2565 - val_loss: 0.6991
Epoch 3/20
52/52 - 0s - 8ms/step - auc: 0.5276 - brier: 0.2526 - loss: 0.6930 - val_auc: 0.5138 - val_brier: 0.2450 - val_loss: 0.6951
Epoch 4/20
52/52 - 0s - 9ms/step - auc: 0.5111 - brier: 0.2515 - loss: 0.6948 - val_auc: 0.4905 - val_brier: 0.2490 - val_loss: 0.6979
Epoch 5/20
52/52 - 0s - 9ms/step - auc: 0.5458 - brier: 0.2515 - loss: 0.6876 - val_auc: 0.5122 - val_brier: 0.2464 - val_loss: 0.6965
Epoch 6/20
52/52 - 0s - 9ms/step - auc: 0.5435 - brier: 0.2503 - loss: 0.6887 - val_auc: 0.4955 - val_brier: 0.2523 - val_loss: 0.7017
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
AAPL: SideSignal.HOLD


In [20]:
symbol = "MCFT"
side, _ = await ML_Pipeline(build_patchtst_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 5s - 104ms/step - auc: 0.5003 - brier: 0.2666 - loss: 0.7358 - val_auc: 0.4795 - val_brier: 0.2597 - val_loss: 0.7135
Epoch 2/20
52/52 - 1s - 10ms/step - auc: 0.5314 - brier: 0.2559 - loss: 0.6953 - val_auc: 0.4941 - val_brier: 0.2645 - val_loss: 0.7223
Epoch 3/20
52/52 - 0s - 9ms/step - auc: 0.5366 - brier: 0.2514 - loss: 0.6901 - val_auc: 0.4512 - val_brier: 0.2692 - val_loss: 0.7220
Epoch 4/20
52/52 - 0s - 8ms/step - auc: 0.5290 - brier: 0.2528 - loss: 0.6936 - val_auc: 0.4721 - val_brier: 0.2659 - val_loss: 0.7199
Epoch 5/20
52/52 - 0s - 8ms/step - auc: 0.5392 - brier: 0.2538 - loss: 0.6928 - val_auc: 0.4667 - val_brier: 0.2627 - val_loss: 0.7131
Epoch 6/20
52/52 - 0s - 8ms/step - auc: 0.5262 - brier: 0.2532 - loss: 0.6951 - val_auc: 0.4504 - val_brier: 0.2791 - val_loss: 0.7601
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
MCFT: SideSignal.HOLD


### GNN-lite

In [21]:
symbol = "GOOG"
side, _ = await ML_Pipeline(build_gnn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 43ms/step - auc: 0.4944 - brier: 0.2587 - loss: 0.7115 - val_auc: 0.5075 - val_brier: 0.2431 - val_loss: 0.6939
Epoch 2/20
52/52 - 0s - 5ms/step - auc: 0.5030 - brier: 0.2563 - loss: 0.7055 - val_auc: 0.4990 - val_brier: 0.2424 - val_loss: 0.6895
Epoch 3/20
52/52 - 0s - 4ms/step - auc: 0.5145 - brier: 0.2546 - loss: 0.6971 - val_auc: 0.5108 - val_brier: 0.2420 - val_loss: 0.6920
Epoch 4/20
52/52 - 0s - 4ms/step - auc: 0.4900 - brier: 0.2525 - loss: 0.6986 - val_auc: 0.5125 - val_brier: 0.2432 - val_loss: 0.6890
Epoch 5/20
52/52 - 0s - 4ms/step - auc: 0.5045 - brier: 0.2513 - loss: 0.6934 - val_auc: 0.5067 - val_brier: 0.2439 - val_loss: 0.6885
Epoch 6/20
52/52 - 0s - 4ms/step - auc: 0.5097 - brier: 0.2512 - loss: 0.6927 - val_auc: 0.4981 - val_brier: 0.2442 - val_loss: 0.6888
Epoch 7/20
52/52 - 0s - 4ms/step - auc: 0.4834 - brier: 0.2510 - loss: 0.6970 - val_auc: 0.4985 - val_brier: 0.2443 - val_loss: 0.6889
Epoch 8/20
52/52 - 0s - 4ms/step - auc: 0.5105 - brier

/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/milad/repositories/Live-Trading-Platform/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control thi

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
GOOG: SideSignal.BUY


In [22]:
symbol = "AAPL"
side, _ = await ML_Pipeline(build_gnn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 44ms/step - auc: 0.5065 - brier: 0.2629 - loss: 0.7187 - val_auc: 0.4777 - val_brier: 0.2667 - val_loss: 0.7119
Epoch 2/20
52/52 - 0s - 5ms/step - auc: 0.4884 - brier: 0.2575 - loss: 0.7121 - val_auc: 0.4937 - val_brier: 0.2581 - val_loss: 0.7200
Epoch 3/20
52/52 - 0s - 4ms/step - auc: 0.4800 - brier: 0.2543 - loss: 0.7095 - val_auc: 0.4790 - val_brier: 0.2578 - val_loss: 0.7182
Epoch 4/20
52/52 - 0s - 4ms/step - auc: 0.5023 - brier: 0.2535 - loss: 0.6987 - val_auc: 0.4758 - val_brier: 0.2517 - val_loss: 0.7063
Epoch 5/20
52/52 - 0s - 4ms/step - auc: 0.5153 - brier: 0.2528 - loss: 0.6948 - val_auc: 0.4875 - val_brier: 0.2514 - val_loss: 0.7004
Epoch 6/20
52/52 - 0s - 4ms/step - auc: 0.5178 - brier: 0.2516 - loss: 0.6940 - val_auc: 0.4678 - val_brier: 0.2534 - val_loss: 0.7028
Epoch 7/20
52/52 - 0s - 4ms/step - auc: 0.5245 - brier: 0.2514 - loss: 0.6917 - val_auc: 0.4875 - val_brier: 0.2495 - val_loss: 0.6979
Epoch 8/20
52/52 - 0s - 4ms/step - auc: 0.5045 - brier

In [23]:
symbol = "MCFT"
side, _ = await ML_Pipeline(build_gnn_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 3s - 58ms/step - auc: 0.5023 - brier: 0.2689 - loss: 0.7442 - val_auc: 0.5018 - val_brier: 0.2732 - val_loss: 0.7301
Epoch 2/20
52/52 - 0s - 6ms/step - auc: 0.5023 - brier: 0.2571 - loss: 0.7099 - val_auc: 0.5047 - val_brier: 0.2599 - val_loss: 0.7057
Epoch 3/20
52/52 - 0s - 5ms/step - auc: 0.5158 - brier: 0.2554 - loss: 0.7008 - val_auc: 0.5044 - val_brier: 0.2690 - val_loss: 0.7093
Epoch 4/20
52/52 - 0s - 5ms/step - auc: 0.5147 - brier: 0.2544 - loss: 0.6992 - val_auc: 0.5102 - val_brier: 0.2735 - val_loss: 0.7117
Epoch 5/20
52/52 - 0s - 4ms/step - auc: 0.5386 - brier: 0.2530 - loss: 0.6929 - val_auc: 0.5052 - val_brier: 0.2755 - val_loss: 0.7121
Epoch 6/20
52/52 - 0s - 4ms/step - auc: 0.5328 - brier: 0.2522 - loss: 0.6923 - val_auc: 0.4747 - val_brier: 0.2704 - val_loss: 0.7201
Epoch 7/20
52/52 - 0s - 4ms/step - auc: 0.5088 - brier: 0.2520 - loss: 0.6974 - val_auc: 0.4653 - val_brier: 0.2771 - val_loss: 0.7353
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
1/1 ━━━━━━━━━━━

### NAD-lite

In [24]:
symbol = "GOOG"
side, _ = await ML_Pipeline(build_autoencoder_classifier_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 39ms/step - auc: 0.5195 - brier: 0.2938 - loss: 1.1125 - val_auc: 0.4905 - val_brier: 0.2420 - val_loss: 1.0624
Epoch 2/20
52/52 - 0s - 4ms/step - auc: 0.4974 - brier: 0.2731 - loss: 0.9221 - val_auc: 0.5063 - val_brier: 0.2530 - val_loss: 1.0221
Epoch 3/20
52/52 - 0s - 3ms/step - auc: 0.4910 - brier: 0.2628 - loss: 0.8690 - val_auc: 0.5006 - val_brier: 0.2493 - val_loss: 0.9797
Epoch 4/20
52/52 - 0s - 3ms/step - auc: 0.5039 - brier: 0.2595 - loss: 0.8163 - val_auc: 0.4985 - val_brier: 0.2511 - val_loss: 0.9364
Epoch 5/20
52/52 - 0s - 3ms/step - auc: 0.5062 - brier: 0.2575 - loss: 0.7875 - val_auc: 0.4934 - val_brier: 0.2521 - val_loss: 0.8876
Epoch 6/20
52/52 - 0s - 3ms/step - auc: 0.5144 - brier: 0.2533 - loss: 0.7697 - val_auc: 0.5126 - val_brier: 0.2587 - val_loss: 0.8731
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
GOOG: SideSignal.BUY


In [25]:
symbol = "AAPL"
side, _ = await ML_Pipeline(build_autoencoder_classifier_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 2s - 38ms/step - auc: 0.5068 - brier: 0.2798 - loss: 1.0987 - val_auc: 0.5083 - val_brier: 0.2517 - val_loss: 0.7551
Epoch 2/20
52/52 - 0s - 4ms/step - auc: 0.5055 - brier: 0.2630 - loss: 0.9345 - val_auc: 0.5238 - val_brier: 0.2491 - val_loss: 0.7344
Epoch 3/20
52/52 - 0s - 3ms/step - auc: 0.4951 - brier: 0.2594 - loss: 0.8942 - val_auc: 0.4973 - val_brier: 0.2503 - val_loss: 0.7393
Epoch 4/20
52/52 - 0s - 3ms/step - auc: 0.5221 - brier: 0.2575 - loss: 0.8567 - val_auc: 0.5119 - val_brier: 0.2483 - val_loss: 0.7413
Epoch 5/20
52/52 - 0s - 3ms/step - auc: 0.5009 - brier: 0.2561 - loss: 0.8375 - val_auc: 0.5013 - val_brier: 0.2483 - val_loss: 0.7457
Epoch 6/20
52/52 - 0s - 3ms/step - auc: 0.5168 - brier: 0.2547 - loss: 0.8098 - val_auc: 0.5065 - val_brier: 0.2462 - val_loss: 0.7436
Epoch 7/20
52/52 - 0s - 3ms/step - auc: 0.5327 - brier: 0.2543 - loss: 0.7898 - val_auc: 0.5181 - val_brier: 0.2459 - val_loss: 0.7482
Epoch 8/20
52/52 - 0s - 3ms/step - auc: 0.5210 - brier

In [26]:
symbol = "MCFT"
side, _ = await ML_Pipeline(build_autoencoder_classifier_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 4s - 74ms/step - auc: 0.4566 - brier: 0.2944 - loss: 1.1468 - val_auc: 0.4940 - val_brier: 0.2541 - val_loss: 0.8632
Epoch 2/20
52/52 - 0s - 4ms/step - auc: 0.5117 - brier: 0.2698 - loss: 0.9243 - val_auc: 0.4646 - val_brier: 0.2632 - val_loss: 0.8315
Epoch 3/20
52/52 - 0s - 3ms/step - auc: 0.5089 - brier: 0.2642 - loss: 0.8618 - val_auc: 0.4665 - val_brier: 0.2662 - val_loss: 0.8174
Epoch 4/20
52/52 - 0s - 3ms/step - auc: 0.4957 - brier: 0.2608 - loss: 0.8439 - val_auc: 0.4936 - val_brier: 0.2715 - val_loss: 0.8061
Epoch 5/20
52/52 - 0s - 3ms/step - auc: 0.5165 - brier: 0.2581 - loss: 0.8139 - val_auc: 0.4640 - val_brier: 0.2668 - val_loss: 0.8000
Epoch 6/20
52/52 - 0s - 3ms/step - auc: 0.4925 - brier: 0.2563 - loss: 0.8104 - val_auc: 0.4714 - val_brier: 0.2636 - val_loss: 0.7870
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
MCFT: SideSignal.HOLD


### CNN-GRU lite

In [27]:
symbol = "GOOG"
side, _ = await ML_Pipeline(build_cnn_gru_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 4s - 71ms/step - auc: 0.5013 - brier: 0.2573 - loss: 0.7077 - val_auc: 0.4762 - val_brier: 0.2499 - val_loss: 0.6938
Epoch 2/20
52/52 - 1s - 20ms/step - auc: 0.5051 - brier: 0.2517 - loss: 0.6966 - val_auc: 0.4722 - val_brier: 0.2586 - val_loss: 0.7023
Epoch 3/20
52/52 - 1s - 19ms/step - auc: 0.5050 - brier: 0.2504 - loss: 0.6939 - val_auc: 0.5137 - val_brier: 0.2709 - val_loss: 0.7204
Epoch 4/20
52/52 - 1s - 19ms/step - auc: 0.5310 - brier: 0.2497 - loss: 0.6877 - val_auc: 0.5081 - val_brier: 0.2735 - val_loss: 0.7220
Epoch 5/20
52/52 - 1s - 19ms/step - auc: 0.5378 - brier: 0.2509 - loss: 0.6916 - val_auc: 0.4790 - val_brier: 0.2900 - val_loss: 0.7530
Epoch 6/20
52/52 - 1s - 21ms/step - auc: 0.4953 - brier: 0.2493 - loss: 0.6930 - val_auc: 0.5300 - val_brier: 0.2808 - val_loss: 0.7315
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
GOOG: SideSignal.BUY


In [28]:
symbol = "AAPL"
side, _ = await ML_Pipeline(build_cnn_gru_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 4s - 85ms/step - auc: 0.4981 - brier: 0.2644 - loss: 0.7250 - val_auc: 0.4984 - val_brier: 0.2539 - val_loss: 0.6961
Epoch 2/20
52/52 - 1s - 21ms/step - auc: 0.5079 - brier: 0.2522 - loss: 0.6983 - val_auc: 0.4675 - val_brier: 0.2509 - val_loss: 0.6964
Epoch 3/20
52/52 - 1s - 20ms/step - auc: 0.5096 - brier: 0.2506 - loss: 0.6935 - val_auc: 0.4923 - val_brier: 0.2488 - val_loss: 0.6934
Epoch 4/20
52/52 - 1s - 19ms/step - auc: 0.5272 - brier: 0.2503 - loss: 0.6903 - val_auc: 0.5131 - val_brier: 0.2505 - val_loss: 0.6922
Epoch 5/20
52/52 - 1s - 20ms/step - auc: 0.5197 - brier: 0.2504 - loss: 0.6912 - val_auc: 0.5093 - val_brier: 0.2520 - val_loss: 0.6926
Epoch 6/20
52/52 - 1s - 20ms/step - auc: 0.5382 - brier: 0.2499 - loss: 0.6889 - val_auc: 0.4999 - val_brier: 0.2499 - val_loss: 0.6929
Epoch 7/20
52/52 - 1s - 20ms/step - auc: 0.5577 - brier: 0.2502 - loss: 0.6857 - val_auc: 0.4960 - val_brier: 0.2495 - val_loss: 0.6932
Epoch 8/20
52/52 - 1s - 19ms/step - auc: 0.5396 

In [29]:
symbol = "MCFT"
side, _ = await ML_Pipeline(build_cnn_gru_lite, symbol, {})
print(f"{symbol}: {side}")

Epoch 1/20
52/52 - 4s - 71ms/step - auc: 0.5109 - brier: 0.2591 - loss: 0.7063 - val_auc: 0.5159 - val_brier: 0.2589 - val_loss: 0.6961
Epoch 2/20
52/52 - 1s - 20ms/step - auc: 0.5028 - brier: 0.2542 - loss: 0.7023 - val_auc: 0.5154 - val_brier: 0.2604 - val_loss: 0.6983
Epoch 3/20
52/52 - 1s - 19ms/step - auc: 0.5200 - brier: 0.2523 - loss: 0.6964 - val_auc: 0.5068 - val_brier: 0.2606 - val_loss: 0.7026
Epoch 4/20
52/52 - 3s - 55ms/step - auc: 0.5197 - brier: 0.2519 - loss: 0.6955 - val_auc: 0.5025 - val_brier: 0.2603 - val_loss: 0.7028
Epoch 5/20
52/52 - 1s - 19ms/step - auc: 0.5389 - brier: 0.2520 - loss: 0.6918 - val_auc: 0.5035 - val_brier: 0.2617 - val_loss: 0.7053
Epoch 6/20
52/52 - 1s - 19ms/step - auc: 0.5522 - brier: 0.2516 - loss: 0.6887 - val_auc: 0.4990 - val_brier: 0.2644 - val_loss: 0.7057
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
MCFT: SideSignal.HOLD
